In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import torch
import sagemaker
from sagemaker.pytorch import PyTorch
import boto3
from io import BytesIO

module_path = os.path.abspath(os.path.join('../py-conjugated/networks/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import morphology_networks as net
import model_training as train
import model_testing as test
import physically_informed_loss_functions as PhysLoss
import network_utils as nuts

torch.manual_seed(28)

In [13]:
bucket_name = 'sagemaker-us-east-2-362637960691'
train_data_location = 'py-conjugated/m2py_labels/OPV_labels/train_set/'
test_data_location = 'py-conjugated/m2py_labels/OPV_labels/test_set/'
model_states_location = 'py-conjugated/model_states/OPV/'

client = boto3.client('s3')
resource = boto3.resource('s3')
s3_bucket = resource.Bucket(bucket_name)

In [14]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [15]:
%autoreload

train_data = nuts.OPV_ImDataset(bucket_name, train_data_location)
test_data = nuts.OPV_ImDataset(bucket_name, test_data_location)

train_data_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = len(train_data), shuffle = True)
test_data_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = len(test_data), shuffle = True)

train_set/
train_set/100C_15min_Sub4_Dev2_labels.npy
train_set/100C_30min_Sub7_Dev4_labels.npy
train_set/100C_30min_Sub7_Dev7_labels.npy
train_set/100C_5min_Sub1_Dev3_labels.npy
train_set/125C_15min_Sub4_Dev1_labels.npy
train_set/125C_15min_Sub4_Dev2_labels.npy
train_set/125C_15min_Sub4_Dev6_labels.npy
train_set/125C_15min_Sub4_postexam_labels.npy
train_set/125C_30min_Sub8_Dev3_labels.npy
train_set/125C_5min_Sub1_Dev2_labels.npy
train_set/125C_5min_Sub1_Dev3_labels.npy
train_set/150C_15min_Sub4_Dev1_labels.npy
train_set/150C_15min_Sub4_Dev7_labels.npy
train_set/150C_30min_Sub7_Dev3_labels.npy
train_set/150C_5min_Sub1_Dev7_labels.npy
train_set/175C_15min_Sub4_Dev2_labels.npy
train_set/175C_15min_Sub4_Dev3_labels.npy
train_set/175C_30min_Sub8_Dev6_labels.npy
train_set/175C_30min_Sub8_Dev8_labels.npy
train_set/175C_5min_Sub2_Dev1_labels.npy
train_set/175C_5min_Sub2_Dev3_labels.npy
train_set/175C_5min_Sub2_postexam_labels.npy
train_set/2018-06-29_P3HT-PCBM_NOANNEAL_S1_D1_labels.npy
train_s

In [31]:
# Hyper parameters
num_epochs = 10
im_learning_rate = 0.008

# Device configuration (GPU if available, otherwise CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [32]:
im_branch_model = net.OPV_m2py_NN(2)

#define the loss function and the optimizer
im_criterion = nn.CrossEntropyLoss()
im_optimizer = torch.optim.Adam(im_branch_model.parameters(), lr = im_learning_rate)


RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 266721280000 bytes. Error code 12 (Cannot allocate memory)


In [ ]:
%autoreload

im_train_epoch_losses = []
im_test_epoch_losses = []

for epoch in range(num_epochs):
    
    # First Train the image branch
    im_train_epoch_loss = train.train_OPV_m2py_model(model = im_branch_model,
                                   training_data_set = im_training_data_set,
                                   criterion = im_criterion,
                                   optimizer = im_optimizer)
    
    im_train_epoch_losses.append(im_train_epoch_loss)
    
    im_test_epoch_loss = test.eval_OPV_m2py_model(model = im_branch_model,
                                 testing_data_set = im_testing_data_set,
                                 criterion = im_criterion)
    
    im_test_epoch_losses.append(im_test_epoch_loss)

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))

epochs = np.arange(1, (num_epochs+1), 1)

plt.plot(epochs, im_train_epoch_losses, c = 'k', label = 'training error')
plt.plot(epochs, im_test_epoch_losses, c = 'r', label = 'testing error')
plt.legend(loc = 'upper right')
plt.show()